In [1]:
with open("file1.txt") as f:
    print(f.readlines())

with open("file2.txt") as f:
    print(f.readlines())

with open("file3.txt") as f:
    print(f.readlines())

['file1 line1\n', 'file1 line2\n', 'file1 line3']
['file2 line1\n', 'file2 line2\n', 'file2 line3']
['file3 line1\n', 'file3 line2\n', 'file3 line3']


In [2]:
with open("file1.txt") as f1, open("file2.txt") as f2:
    print(f1.readlines())
    print(f2.readlines())

['file1 line1\n', 'file1 line2\n', 'file1 line3']
['file2 line1\n', 'file2 line2\n', 'file2 line3']


In [3]:
with open("file1.txt") as f1:
    with open("file2.txt") as f2:
        with open("file3.txt") as f3:
            print(f1.readlines())
            print(f2.readlines())
            print(f3.readlines())

['file1 line1\n', 'file1 line2\n', 'file1 line3']
['file2 line1\n', 'file2 line2\n', 'file2 line3']
['file3 line1\n', 'file3 line2\n', 'file3 line3']


In [4]:
# what if we don't know how many files we have? 

In [5]:
from contextlib import contextmanager


@contextmanager
def open_file(fname):
    print(f"opening {fname}")
    f = open(fname)
    try:
        yield f
    finally:
        print(f"closing {fname}")
        f.close()

In [6]:
f_names = "file1.txt", "file2.txt", "file3.txt"
exits = []
enters = []

for f_name in f_names:
    ctx = open_file(f_name)  # creates context object (but doesn't fires the __enter__ yet)
    enters.append(ctx.__enter__)
    exits.append(ctx.__exit__)

In [7]:
files = [enter() for enter in enters]

opening file1.txt
opening file2.txt
opening file3.txt


In [8]:
while True:
    try:
        rows = [next(f).strip("\n") for f in files]
    except StopIteration:
        break
    else:
        row = ", ".join(rows)
        print(row)

file1 line1, file2 line1, file3 line1
file1 line2, file2 line2, file3 line2
file1 line3, file2 line3, file3 line3


In [9]:
for context_exit in exits[::-1]:
    context_exit(None, None, None)

closing file3.txt
closing file2.txt
closing file1.txt


In [10]:
# variable number of files, eg. taken from scrapping the dir
f_names = "file1.txt", "file2.txt", "file3.txt"  
exits = []
enters = []

# creating context managers
for f_name in f_names:
    ctx = open_file(f_name)  # creates context object (but doesn't fires the __enter__ yet)
    enters.append(ctx.__enter__)
    exits.append(ctx.__exit__)

# entering context managers
files = [enter() for enter in enters]

# doing the work
while True:
    try:
        rows = [next(f).strip("\n") for f in files]
    except StopIteration:
        # will stop as soon as shortest file is exhausted
        break
    else:
        row = ", ".join(rows)
        print(row)

# exiting context managers
for context_exit in exits[::-1]:
    context_exit(None, None, None)

opening file1.txt
opening file2.txt
opening file3.txt
file1 line1, file2 line1, file3 line1
file1 line2, file2 line2, file3 line2
file1 line3, file2 line3, file3 line3
closing file3.txt
closing file2.txt
closing file1.txt


In [11]:
# context manager that manages context managers :) 

class NestedContests:
    def __init__(self, *contexts):
        self._enters = []
        self._exits = []
        self._values = []

        for ctx in contexts:
            self._enters.append(ctx.__enter__)
            self._exits.append(ctx.__exit__)

    def __enter__(self):
        for enter in self._enters:
            self._values.append(enter())
        return self._values

    def __exit__(self, exc_type, exc_value, exc_tb):
        for exit in self._exits[::-1]:
            exit(exc_type, exc_value, exc_tb)
        return False


In [12]:
with NestedContests(
    open_file("file1.txt"),
    open_file("file2.txt"),
    open_file("file3.txt"),
) as files:
    while True:
        try:
            rows = [next(f).strip("\n") for f in files]
        except StopIteration:
            # will stop as soon as shortest file is exhausted
            break
        else:
            row = ", ".join(rows)
            print(row)

opening file1.txt
opening file2.txt
opening file3.txt
file1 line1, file2 line1, file3 line1
file1 line2, file2 line2, file3 line2
file1 line3, file2 line3, file3 line3
closing file3.txt
closing file2.txt
closing file1.txt


In [13]:
file_names = ("file1.txt", "file2.txt", "file3.txt")
contexts = [open_file(fname) for fname in file_names]

with NestedContests(*contexts) as files:
    print("do the work")

opening file1.txt
opening file2.txt
opening file3.txt
do the work
closing file3.txt
closing file2.txt
closing file1.txt


In [14]:
# version with dynamic number of contexts
class NestedContexts:
    def __init__(self):
        self._exits = []

    def __enter__(self):
        return self

    def enter_context(self, ctx):
        self._exits.append(ctx.__exit__)
        value = ctx.__enter__()
        return value

    def __exit__(self, exc_type, exc_value, exc_tb):
        for exit in self._exits[::-1]:
            exit(exc_type, exc_value, exc_tb)
        return False


In [15]:
file_names = ("file1.txt", "file2.txt", "file3.txt")
with NestedContexts() as context_stack:
    files = [context_stack.enter_context(open_file(f)) for f in file_names]
    print("do the work on files")


opening file1.txt
opening file2.txt
opening file3.txt
do the work on files
closing file3.txt
closing file2.txt
closing file1.txt


In [16]:
# this nested stack behaviour is already implemented in python :)
from contextlib import ExitStack


help(ExitStack)

Help on class ExitStack in module contextlib:

class ExitStack(_BaseExitStack, AbstractContextManager)
 |  Context manager for dynamic management of a stack of exit callbacks.
 |
 |  For example:
 |      with ExitStack() as stack:
 |          files = [stack.enter_context(open(fname)) for fname in filenames]
 |          # All opened files will automatically be closed at the end of
 |          # the with statement, even if attempts to open files later
 |          # in the list raise an exception.
 |
 |  Method resolution order:
 |      ExitStack
 |      _BaseExitStack
 |      AbstractContextManager
 |      abc.ABC
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __enter__(self)
 |      Return `self` upon entering the runtime context.
 |
 |  __exit__(self, *exc_details)
 |      Raise any exception triggered within the runtime context.
 |
 |  close(self)
 |      Immediately unwind the context stack.
 |
 |  ----------------------------------------------------------------------
 |

In [18]:
from contextlib import ExitStack

file_names = ("file1.txt", "file2.txt", "file3.txt")
with ExitStack() as context_stack:
    files = [context_stack.enter_context(open(f)) for f in file_names]
    print("do the work on files")

do the work on files
